In [1]:
import os
from dotenv import load_dotenv

# access the openai api and unstract llm whisperer key from .env file and map it to the current system
os.environ['OPENAI_API_KEY']=os.getenv("openai_api_key")

In [3]:
%pip install -q tabula-py

Note: you may need to restart the kernel to use updated packages.


In [4]:
import tabula
import pandas as pd

def extract_tables_from_pdf(pdf_path: str, pages: str = 'all', guess: bool = True) -> list:
    """
    Extract tables from PDF using tabula-py
    
    Args:
        pdf_path: Path to PDF file
        pages: Page numbers to extract (e.g., '1-3', '1,3,5', 'all')
        guess: Whether to guess table structure (True is usually better for financial tables)
    
    Returns:
        List of pandas DataFrames containing tables
    """
    try:
        # Extract tables
        tables = tabula.read_pdf(
            pdf_path,
            pages=pages,
            guess=guess,
            multiple_tables=True,  # Extract multiple tables per page
            pandas_options={'header': None}  # Don't assume first row is header
        )
        
        print(f"Found {len(tables)} tables")
        return tables
    
    except Exception as e:
        print(f"Error extracting tables: {str(e)}")
        return []

def find_ebitda_tables(tables: list) -> list:
    """
    Find tables that look like EBITDA reconciliations
    """
    ebitda_tables = []
    
    # Keywords that might indicate EBITDA tables
    keywords = ['ebitda', 'earnings', 'reconciliation', 'adjusted']
    
    for i, table in enumerate(tables):
        # Convert table contents to string for searching
        table_str = table.to_string().lower()
        
        # Check if any keywords are in the table
        if any(keyword in table_str for keyword in keywords):
            print(f"Found potential EBITDA table (Table {i+1})")
            ebitda_tables.append(table)
    
    return ebitda_tables

# Extract tables from specific pages (adjust page range based on where EBITDA tables are)
pdf_path = "../raw-docs/uber-10k-report.pdf"
tables = extract_tables_from_pdf(
    pdf_path,
    pages='1-2'  # Adjust this range based on where EBITDA tables are
)

# Look for EBITDA tables
ebitda_tables = find_ebitda_tables(tables)

# Print all tables for inspection
print("\nAll extracted tables:")
for i, table in enumerate(tables):
    print(f"\nTable {i+1}:")
    print(table)
    print("-" * 50)

# # Save tables to Excel (each table on a different sheet)
# if tables:
#     with pd.ExcelWriter('uber_10k_tables.xlsx') as writer:
#         for i, table in enumerate(tables):
#             sheet_name = f'Table_{i+1}'
#             table.to_excel(writer, sheet_name=sheet_name, index=False)
#     print("\nTables saved to uber_10k_tables.xlsx")

# Print EBITDA tables specifically
if ebitda_tables:
    print("\nEBITDA Tables found:")
    for i, table in enumerate(ebitda_tables):
        print(f"\nEBITDA Table {i+1}:")
        print(table)
        print("-" * 50)
else:
    print("\nNo EBITDA tables found")

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


Found 1 tables
Found potential EBITDA table (Table 1)

All extracted tables:

Table 1:
                                                    0    1        2    3  \
0                                       (In millions)  NaN     2022  NaN   
1                     Adjusted EBITDA reconciliation:  NaN      NaN  NaN   
2   Net income (loss) attributable to Uber Technol...    $  (9,141)    $   
3                                       Add (deduct):  NaN      NaN  NaN   
4   Net income attributable to non-controlling int...  NaN        3  NaN   
5           Provision for (benefit from) income taxes  NaN    (181)  NaN   
6               Income from equity method investments  NaN    (107)  NaN   
7                                    Interest expense  NaN      565  NaN   
8                         Other (income) expense, net  NaN    7,029  NaN   
9                       Depreciation and amortization  NaN      947  NaN   
10                   Stock-based compensation expense  NaN    1,793  NaN   
1

In [5]:
# let's use langchain to extract the data from the tables
import pandas as pd
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Optional
import json

# Pydantic model for EBITDA data
class EBITDAItem(BaseModel):
    description: str = Field(description="Description of the line item")
    value_2022: float = Field(description="Value for year 2022 in millions")
    value_2023: float = Field(description="Value for year 2023 in millions")

class EBITDAReconciliation(BaseModel):
    items: List[EBITDAItem] = Field(description="List of items in the EBITDA reconciliation")
    total_adjusted_ebitda_2022: float = Field(description="Total Adjusted EBITDA for 2022")
    total_adjusted_ebitda_2023: float = Field(description="Total Adjusted EBITDA for 2023")

def process_ebitda_table(table: pd.DataFrame) -> str:
    """Convert table to structured string for LLM processing"""
    return table.to_string()

def extract_ebitda_data(table_text: str) -> dict:
    """Extract EBITDA data using LangChain"""
    
    # Create the prompt
    system_template = """You are a financial analyst extracting data from an EBITDA reconciliation table.
    The table shows values for 2022 and 2023. Extract all line items and their values.
    Treat numbers in parentheses as negative values.
    Convert all values to millions of dollars."""
    
    human_template = """Here's the EBITDA reconciliation table:
    {table_text}
    
    Please extract the data in the specified format. Ensure all numbers are in millions and properly signed (negative for numbers in parentheses)."""
    
    # Create the chat prompt
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    
    parser = PydanticOutputParser(pydantic_object=EBITDAReconciliation)
    chat_prompt = ChatPromptTemplate.from_messages([
        system_message_prompt,
        human_message_prompt
    ])
    
    # Format the prompt
    prompt = chat_prompt.format_prompt(
        table_text=table_text,
        format_instructions=parser.get_format_instructions()
    ).to_messages()
    
    # Get response from LLM
    llm = ChatOpenAI(temperature=0)
    response = llm.invoke(prompt)
    
    return response.content



In [6]:
def parse_ebitda_text_to_json(text: str) -> dict:
    """
    Convert EBITDA text output to structured JSON
    """
    # Initialize the structure
    ebitda_data = {
        "items": [],
        "total_adjusted_ebitda_2022": None,
        "total_adjusted_ebitda_2023": None
    }
    
    # Split text into lines and process each line
    lines = text.strip().split('\n')
    current_item = None
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # Check if this is a new item (starts with -)
        if line.startswith('-'):
            current_item = {
                "description": line.strip('- :'),
                "value_2022": None,
                "value_2023": None
            }
            ebitda_data["items"].append(current_item)
        
        # Check if this is a year value
        elif '2022:' in line:
            value = float(line.split(':')[1].strip())
            if current_item:
                current_item["value_2022"] = value
            # Check if this is the Adjusted EBITDA total
            if "Adjusted EBITDA" in current_item["description"]:
                ebitda_data["total_adjusted_ebitda_2022"] = value
                
        elif '2023:' in line:
            value = float(line.split(':')[1].strip())
            if current_item:
                current_item["value_2023"] = value
            # Check if this is the Adjusted EBITDA total
            if "Adjusted EBITDA" in current_item["description"]:
                ebitda_data["total_adjusted_ebitda_2023"] = value
    
    return ebitda_data

# Use the function
extracted_data = extract_ebitda_data(table)
parsed_json = parse_ebitda_text_to_json(extracted_data)

# Print the structured JSON
print(json.dumps(parsed_json, indent=2))

# Save to file
with open("../saved-docs/pypdf_ebitda_data.json", "w") as f:
    json.dump(parsed_json, f, indent=2)


{
  "items": [
    {
      "description": "Net income (loss) attributable to Uber Technologies, Inc.",
      "value_2022": -9.141,
      "value_2023": 1.887
    },
    {
      "description": "Net income attributable to non-controlling interests",
      "value_2022": 3.0,
      "value_2023": 0.269
    },
    {
      "description": "Provision for (benefit from) income taxes",
      "value_2022": -0.181,
      "value_2023": 0.213
    },
    {
      "description": "Income from equity method investments",
      "value_2022": -0.107,
      "value_2023": -0.048
    },
    {
      "description": "Interest expense",
      "value_2022": 0.565,
      "value_2023": 0.633
    },
    {
      "description": "Other (income) expense, net",
      "value_2022": 7.029,
      "value_2023": -1.844
    },
    {
      "description": "Depreciation and amortization",
      "value_2022": 0.947,
      "value_2023": 0.823
    },
    {
      "description": "Stock-based compensation expense",
      "value_2022": 1.7

In [7]:
# open the extracted data from the file
with open("../saved-docs/pypdf_ebitda_data.json", "r") as f:
    parsed_data = json.load(f)
    
# create the ask_ebitda_question function
# Helper function to ask questions about the EBITDA data
def ask_ebitda_question(question: str, parsed_data: dict) -> str:
    """Ask questions about the extracted EBITDA data"""
    
    prompt = f"""Based on the following EBITDA data:
    {json.dumps(parsed_data, indent=2)}
    
    Please answer this question: {question}
    
    Provide a clear, concise answer with specific numbers when relevant."""
    
    llm = ChatOpenAI(temperature=0)
    response = llm.invoke(prompt)
    return response.content

# Example questions
questions = [
    "What was Uber's Adjusted EBITDA in 2023?",
    "What was Uber's Adjusted EBITDA in 2022?",
    "What was the year-over-year change in Adjusted EBITDA?",
    "What were the major components affecting EBITDA in 2023?"
]

# Only proceed with questions if we have data
if parsed_data:
    # Ask questions about the extracted data
    print("\nAnswering Questions about EBITDA:")
    for question in questions:
        print(f"\nQ: {question}")
        answer = ask_ebitda_question(question, parsed_data)
        print(f"A: {answer}")
else:
    print("\nNo data available for questions")


Answering Questions about EBITDA:

Q: What was Uber's Adjusted EBITDA in 2023?
A: Uber's Adjusted EBITDA in 2023 was $4.052 billion.

Q: What was Uber's Adjusted EBITDA in 2022?
A: Uber's Adjusted EBITDA in 2022 was $1.713 billion.

Q: What was the year-over-year change in Adjusted EBITDA?
A: The year-over-year change in Adjusted EBITDA was an increase of $2.339 billion from 2022 to 2023.

Q: What were the major components affecting EBITDA in 2023?
A: The major components affecting EBITDA in 2023 were:
1. Increase in Net income attributable to Uber Technologies, Inc. from -9.141 in 2022 to 1.887 in 2023, contributing positively to EBITDA.
2. Decrease in Other (income) expense, net from 7.029 in 2022 to -1.844 in 2023, contributing positively to EBITDA.
3. Increase in Stock-based compensation expense from 1.793 in 2022 to 1.935 in 2023, contributing negatively to EBITDA.
4. Increase in Legal, tax, and regulatory reserve changes and settlements from 0.732 in 2022 to 0.009 in 2023, contr